In [27]:
!export CUDA_VISIBLE_DEVICES=5

4970.25s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


In [28]:
import os
import cv2
import sys
import numpy as np
from PIL import Image
from pathlib import Path
from paddle.utils import try_import

parent_dir = Path().resolve().parent
# # 插入模块路径
sys.path.insert(0, str(parent_dir))

from paddleocr import PPStructure, save_structure_res, PaddleOCR, draw_structure_result
from paddleocr.ppstructure.recovery.recovery_to_doc import (
    sorted_layout_boxes,
    convert_info_docx,
)


ocr_engine = PPStructure(recovery=True, structure_version="PP-StructureV2")

save_folder = "./layout_res"
pdf_path = "../data/basic.pdf"
font_path = "../fonts/simfang.ttf"  # PaddleOCR下提供字体包

[2024/06/06 13:16:00] ppocr DEBUG: Namespace(help='==SUPPRESS==', use_gpu=False, use_xpu=False, use_npu=False, use_mlu=False, ir_optim=True, use_tensorrt=False, min_subgraph_size=15, precision='fp32', gpu_mem=500, gpu_id=0, image_dir=None, page_num=0, det_algorithm='DB', det_model_dir='/Users/liangzhu/.paddleocr/whl/det/ch/ch_PP-OCRv4_det_infer', det_limit_side_len=960, det_limit_type='max', det_box_type='quad', det_db_thresh=0.3, det_db_box_thresh=0.6, det_db_unclip_ratio=1.5, max_batch_size=10, use_dilation=False, det_db_score_mode='fast', det_east_score_thresh=0.8, det_east_cover_thresh=0.1, det_east_nms_thresh=0.2, det_sast_score_thresh=0.5, det_sast_nms_thresh=0.2, det_pse_thresh=0, det_pse_box_thresh=0.85, det_pse_min_area=16, det_pse_scale=1, scales=[8, 16, 32], alpha=1.0, beta=1.0, fourier_degree=5, rec_algorithm='SVTR_LCNet', rec_model_dir='/Users/liangzhu/.paddleocr/whl/rec/ch/ch_PP-OCRv4_rec_infer', rec_image_inverse=True, rec_image_shape='3, 48, 320', rec_batch_num=6, max_t

In [29]:
# 从 PDF 中获取页面图像
def covert_pdf_to_img(pdf_path):
    fitz = try_import("fitz")
    imgs = []

    with fitz.open(pdf_path) as pdf:
        for pg in range(0, pdf.page_count):
            page = pdf[pg]
            pm = page.get_pixmap(alpha=False)
            mat = fitz.Matrix(
                2, 2
            )  # 将 PDF 页面缩放两倍，为了提高图像的分辨率和清晰度。
            pm = page.get_pixmap(matrix=mat, alpha=False)

            # if width or height > 2000 pixels, don't enlarge the image
            if pm.width > 2000 or pm.height > 2000:
                pm = page.get_pixmap(matrix=fitz.Matrix(1, 1), alpha=False)

            # 将像素图转换为 PIL 图像对象。
            img = Image.frombytes("RGB", [pm.width, pm.height], pm.samples)

            # 将 PIL 图像转换为 NumPy 数组，并从 RGB 格式转换为 BGR 格式，以便 OpenCV 使用。
            img = cv2.cvtColor(np.array(img), cv2.COLOR_RGB2BGR)
            imgs.append(img)

    return imgs

In [30]:
imgs = covert_pdf_to_img(pdf_path)
print(len(imgs))

2


获取页面的长宽像素。

print(page.bound().height)

print(page.bound().width)

In [31]:
def get_layout(imgs, ocr_engine):
    # for index, img in enumerate(imgs):
    index = 0
    img = imgs[0]
    result = ocr_engine(img)
    save_structure_res(
        result, save_folder, os.path.basename(pdf_path).split(".")[0], index
    )
    print(result)
    h, w, _ = img.shape
    res = sorted_layout_boxes(result, w)
    print("===")
    print(res)
    convert_info_docx(img, res, save_folder, os.path.basename(pdf_path).split(".")[0])

In [33]:
get_layout(imgs, ocr_engine)

[2024/06/06 14:09:08] ppocr DEBUG: dt_boxes num : 64, elapsed : 0.6419308185577393
[2024/06/06 14:09:19] ppocr DEBUG: rec_res num  : 64, elapsed : 11.2829430103302
[2024/06/06 14:09:20] ppocr DEBUG: dt_boxes num : 27, elapse : 0.15581011772155762
[2024/06/06 14:09:23] ppocr DEBUG: rec_res num  : 27, elapse : 3.0825719833374023
[{'type': 'text', 'bbox': [646, 770, 1049, 969], 'img': array([[[255, 255, 255],
        [255, 255, 255],
        [255, 255, 255],
        ...,
        [255, 255, 255],
        [255, 255, 255],
        [255, 255, 255]],

       [[255, 255, 255],
        [255, 255, 255],
        [255, 255, 255],
        ...,
        [255, 255, 255],
        [255, 255, 255],
        [255, 255, 255]],

       [[255, 255, 255],
        [255, 255, 255],
        [255, 255, 255],
        ...,
        [255, 255, 255],
        [255, 255, 255],
        [255, 255, 255]],

       ...,

       [[255, 255, 255],
        [255, 255, 255],
        [255, 255, 255],
        ...,
        [255, 255, 

##### draw_structure_result

In [6]:
for index, img in enumerate(imgs):
    result = ocr_engine(img)
    save_structure_res(
        result, save_folder, os.path.basename(pdf_path).split(".")[0], index
    )
    h, w, _ = img.shape
    res = sorted_layout_boxes(result, w)
    convert_info_docx(img, res, save_folder, os.path.basename(pdf_path).split(".")[0])

    # 保存每一页的结构化结果图片
    im_show = draw_structure_result(
        Image.fromarray(cv2.cvtColor(img, cv2.COLOR_BGR2RGB)),
        result,
        font_path=font_path,
    )
    im_show = Image.fromarray(im_show)
    im_show.save(os.path.join(save_folder, f"result_page_{index + 1}.jpg"))

[2024/06/06 10:47:42] ppocr DEBUG: dt_boxes num : 64, elapsed : 0.057605743408203125
[2024/06/06 10:47:42] ppocr DEBUG: rec_res num  : 64, elapsed : 0.2067699432373047
[2024/06/06 10:47:43] ppocr DEBUG: dt_boxes num : 27, elapse : 0.025567054748535156
[2024/06/06 10:47:43] ppocr DEBUG: rec_res num  : 27, elapse : 0.07384419441223145
[2024/06/06 10:47:43] ppocr INFO: docx save to ./layout_res/basic_ocr.docx
[2024/06/06 10:47:43] ppocr DEBUG: dt_boxes num : 11, elapsed : 0.04267549514770508
[2024/06/06 10:47:43] ppocr DEBUG: rec_res num  : 11, elapsed : 0.061767578125
[2024/06/06 10:47:43] ppocr INFO: docx save to ./layout_res/basic_ocr.docx
